In [25]:
import pandas as pd

ENERGY_DATASET_PATH = "../dataset/energy_dataset.csv"
WEATHER_DATASET_PATH = "../dataset/weather_features.csv"

energy_dataset = pd.read_csv(ENERGY_DATASET_PATH)
weather_dataset = pd.read_csv(WEATHER_DATASET_PATH)

In [26]:
# show statistics of the energy dataset
print(energy_dataset.describe())


       generation biomass  generation fossil brown coal/lignite  \
count        35045.000000                          35046.000000   
mean           383.513540                            448.059208   
std             85.353943                            354.568590   
min              0.000000                              0.000000   
25%            333.000000                              0.000000   
50%            367.000000                            509.000000   
75%            433.000000                            757.000000   
max            592.000000                            999.000000   

       generation fossil coal-derived gas  generation fossil gas  \
count                             35046.0           35046.000000   
mean                                  0.0            5622.737488   
std                                   0.0            2201.830478   
min                                   0.0               0.000000   
25%                                   0.0            412

In [27]:
print(weather_dataset.describe())

                temp       temp_min       temp_max      pressure  \
count  178396.000000  178396.000000  178396.000000  1.783960e+05   
mean      289.618605     288.330442     291.091267  1.069261e+03   
std         8.026199       7.955491       8.612454  5.969632e+03   
min       262.240000     262.240000     262.240000  0.000000e+00   
25%       283.670000     282.483602     284.650000  1.013000e+03   
50%       289.150000     288.150000     290.150000  1.018000e+03   
75%       295.150000     293.730125     297.150000  1.022000e+03   
max       315.600000     315.150000     321.150000  1.008371e+06   

            humidity    wind_speed       wind_deg        rain_1h  \
count  178396.000000  178396.00000  178396.000000  178396.000000   
mean       68.423457       2.47056     166.591190       0.075492   
std        21.902888       2.09591     116.611927       0.398847   
min         0.000000       0.00000       0.000000       0.000000   
25%        53.000000       1.00000      55.0000

## Combination and Drop of Columns

In [28]:
weather = weather_dataset.drop(['temp_min', 'temp_max', 'pressure', 'humidity', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h', 'weather_id'], axis=1)
print(weather.describe())   


                temp    wind_speed     clouds_all
count  178396.000000  178396.00000  178396.000000
mean      289.618605       2.47056      25.073292
std         8.026199       2.09591      30.774129
min       262.240000       0.00000       0.000000
25%       283.670000       1.00000       0.000000
50%       289.150000       2.00000      20.000000
75%       295.150000       4.00000      40.000000
max       315.600000     133.00000     100.000000


In [29]:
# Load the dataset
df = energy_dataset

# Combine fossil fuel-based generation
fossil_fuels = [
    "generation fossil brown coal/lignite", "generation fossil coal-derived gas",
    "generation fossil gas", "generation fossil hard coal", "generation fossil oil",
    "generation fossil oil shale", "generation fossil peat"
]
df["fossil fuels combined"] = df[fossil_fuels].sum(axis=1)

# Rename and aggregate renewable sources
df["windpower"] = df[["generation wind offshore", "generation wind onshore"]].sum(axis=1)
df["windpower forecast"] = df[["forecast wind offshore eday ahead", "forecast wind onshore day ahead"]].sum(axis=1)
df["solarpower"] = df["generation solar"]
df["solarpower forecast"] = df["forecast solar day ahead"]

# Aggregate other green energy sources (excluding wind and solar)
green_energy_sources = [
    "generation biomass", "generation geothermal", "generation hydro pumped storage aggregated",
    "generation hydro run-of-river and poundage", "generation hydro water reservoir",
    "generation marine", "generation other", "generation other renewable", "generation waste"
]
df["other green energy"] = df[green_energy_sources].sum(axis=1)

# Select relevant columns
columns_to_keep = [
    "time", "fossil fuels combined", "windpower", "windpower forecast", "solarpower", "solarpower forecast",
    "other green energy", "total load forecast", "total load actual", "price day ahead", "price actual"
]
df = df[columns_to_keep]

# Save the processed dataset
df.to_csv("simplified_dataset.csv", index=False)

# Display the first few rows
print(df.head())



                        time  fossil fuels combined  windpower  \
0  2015-01-01 00:00:00+01:00                10156.0     6378.0   
1  2015-01-01 01:00:00+01:00                10437.0     5890.0   
2  2015-01-01 02:00:00+01:00                 9918.0     5461.0   
3  2015-01-01 03:00:00+01:00                 8859.0     5238.0   
4  2015-01-01 04:00:00+01:00                 8313.0     4935.0   

   windpower forecast  solarpower  solarpower forecast  other green energy  \
0              6436.0        49.0                 17.0              3709.0   
1              5856.0        50.0                 16.0              3425.0   
2              5454.0        50.0                  8.0              3104.0   
3              5151.0        50.0                  2.0              2475.0   
4              4861.0        42.0                  9.0              2407.0   

   total load forecast  total load actual  price day ahead  price actual  
0              26118.0            25385.0            50.10 

In [30]:
import numpy as np

# set zero values to NaN
df = pd.read_csv("simplified_dataset.csv")
df = df.replace(0.0, np.nan)

# count the number of missing values
print(df.isnull().sum())


# interpolate using pandas
df = df.interpolate()

df.to_csv("interpolated_dataset.csv", index=False)


time                       0
fossil fuels combined     18
windpower                 21
windpower forecast         0
solarpower                21
solarpower forecast      539
other green energy        21
total load forecast        0
total load actual         36
price day ahead            0
price actual               0
dtype: int64


/var/folders/z1/rghddhpx3csdrmp3zc65yc1w0000gn/T/ipykernel_1892/1062022161.py:12: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate()


## Clean data
There are columns with missing values, so we should interpolate them.